<h1>Libraries and Data Base</h1>

In [ ]:
import numpy as np
from sklearn.cross_validation import train_test_split
from IPython import display
from sklearn.metrics import roc_curve, auc, confusion_matrix
from __future__ import division
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import matplotlib.pylab as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import tensorflow as tf
%matplotlib inline


#Labeled Data base
filename = 'DataBases/LBPTruncadoRaw4.csv'
csv = np.genfromtxt (filename, delimiter=',')
Y = csv[:,0]
X = csv[:,1:]
print "Labels:"
print Y.shape
print "Data:"
print X.shape
print ""

<h1>Dimensionality Reduction</h1>

In [ ]:
# Data normalization

#data_scaled = scale(X, axis=0, with_mean=False, with_std=True)
#print "Datos Normalizados"
#print data_scaled.shape

#PCA projection
#pca = PCA(n_components=500)
#X_new = pca.fit_transform(data_scaled)

print "Data:"
print X.shape

#Dimensionality Reduction with Random Forest model
clf = RandomForestClassifier()
clf = clf.fit(X, Y)
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)

print "New data:"
print X_new.shape

<h1>Data base partition</h1>

In [ ]:
#Decide wheter reduce dimensionality (if yes, run the previous cell beforehand)
dimRed = True

#size of datasets
trainSize = 0.6
valiSize = 0.1
#the test size is not given because it is fixed with the previous two fractions

if dimRed:
    data = X_new
else:
    data = X

#Split
X_train, X_aux, Y_train, Y_aux = train_test_split(data[350:,:], Y[350:], train_size=trainSize) #balanced
#X_train, X_aux, Y_train, Y_aux = train_test_split(data, Y, train_size=trainSize) #original
X_vali, X_test, Y_vali, Y_test = train_test_split(X_aux, Y_aux, train_size= valiSize/(1-trainSize))

print "Training set:"
print "Labels:"
print Y_train.shape
print "Data:"
print X_train.shape
men = 0
women = 0
for i in range(len(Y_train)):
    if (Y_train[i]==1):
        women = women + 1
    else:
        men = men + 1
print "Men in training set:"
print men
print "Women in training set:"
print women              

print ""
print "Validation set:"
print "Labels:"
print Y_vali.shape
print "Data:"
print X_vali.shape
men = 0
women = 0
for i in range(len(Y_vali)):
    if (Y_vali[i]==1):
        women = women + 1
    else:
        men = men + 1
print "Men in training set:"
print men
print "Women in training set:"
print women  

print ""
print "Test set:"
print "Labels:"
print Y_test.shape
print "Data:"
print X_test.shape
men = 0
women = 0
for i in range(len(Y_test)):
    if (Y_test[i]==1):
        women = women + 1
    else:
        men = men + 1
print "Men in training set:"
print men
print "Women in training set:"
print women  

print ""

<h1>LDA Classifier</h1>

In [ ]:
#LDA with the entire dataset
classifier = LinearDiscriminantAnalysis()
classifier.fit(X, Y)
data_projected = classifier.transform(X)

#Plotting the result
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(range(len(X)) , data_projected, c=Y, linewidth=0, alpha=0.5, s=20)
plt.xlim([0,len(X)-1])
plt.xlabel("Data Instance")
plt.ylabel("LDA Projection")
plt.grid()
plt.title('LDA Projection, Entire Set')



#Test the LDA as a classifier
#For this, is important to train the LDA correctly, that is, with the X_train dataset only.

#print 'Training results:'
#predicted_training_labels = classifier.predict_proba(X_train)[:,1]
#[[VN,FP],[FN,VP]]=confusion_matrix(Y_train.astype(bool),(predicted_training_labels>0.5).astype(bool)).astype(float)
#print 'VP: %d, VN: %d, FP: %d, FN: %d' %(VP,VN,FP,FN)
#print 'Porcentaje de clasificaciones correctas: %%%f' %(100.0*(VP+VN)/(VP+VN+FP+FN))
#print 'Precision: %%%f' %(100.0*VP/(VP+FP))
#print 'Recall: %%%f' %(100.0*VP/(VP+FN))
#print ''

#print 'Test results:'
#predicted_test_labels = classifier.predict_proba(X_test)[:,1]
#[[VN,FP],[FN,VP]]=confusion_matrix(Y_test.astype(bool),(predicted_test_labels>0.5).astype(bool)).astype(float)
#print 'VP: %d, VN: %d, FP: %d, FN: %d' %(VP,VN,FP,FN)
#print 'Porcentaje de clasificaciones correctas: %%%f' %(100.0*(VP+VN)/(VP+VN+FP+FN))
#print 'Precision: %%%f' %(100.0*VP/(VP+FP))
#print 'Recall: %%%f' %(100.0*VP/(VP+FN))
#print ''

#fpr, tpr, th = roc_curve(Y_test, predicted_test_labels)
    
#plt.figure(figsize=(10, 4))
#plt.subplot2grid((1,2),(0,0))
#plt.plot(fpr, tpr, linewidth=2, alpha=0.5)
#plt.xlabel("False Positive Rate")
#plt.ylabel("True Positive Rate")
#plt.grid()
#plt.title('ROC Curve')

#plt.subplot2grid((1,2),(0,1))
#plt.plot(fpr, 1.0 - tpr, linewidth=2, alpha=0.5)
#plt.xlabel("False Positive Rate")
#plt.ylabel("False Negative Rate")
#plt.yscale('log')
#plt.xscale('log')
#plt.grid()
#plt.title('DET Curve')


<h1>Two Layer MLP Configuration</h1>

In [ ]:
#-----------------------
#PARAMETERS OF INICIALIZATION
#-----------------------

# Training parameters
minibatch_size = 8
n_epochs = 2000
n_training_samples = len(Y_train)
target_cost = 0.01

#Gradient Descent parameters
learning_rate = 0.1
momentum = 0.1

#Number of neurons
n_inputs = len(X_train[0])
n_hidden = 15
n_classes = 2
n_neurons = np.hstack((n_inputs,n_hidden,n_classes)).astype(int)

print "Number of inputs:"
print n_inputs

#-----------------------
#CONSTRUCTION OF THE MLP
#-----------------------

MLP_input = tf.placeholder(tf.float32, shape=[None, n_inputs])
previous_layer = MLP_input

# Pointer to weights and biases for regularization
regularizers = 0

# Iteration constructing one layer at a time
for level in range(len(n_neurons)-1):
    weights = tf.Variable(tf.random_uniform   ([n_neurons[level],n_neurons[level+1]], minval= -np.sqrt(3/n_neurons[level]),maxval=np.sqrt(3/n_neurons[level])))
    #weights = tf.Variable(tf.truncated_normal([n_neurons[level],n_neurons[level+1]], stddev=0.1))
    biases = tf.Variable(tf.constant(0., shape=[n_neurons[level+1]]))
    applied_weights = tf.matmul(previous_layer, weights) + biases

    regularizers += tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)

    if level < len(n_neurons)-2:
        layer = tf.sigmoid(applied_weights)
        previous_layer = layer
    else:
        MLP = tf.nn.softmax(applied_weights)
        break
        
#-----------------------
#COST FUNCTION
#-----------------------

# Accomodate target
target = tf.placeholder(tf.float32, shape=[None])
one_hot_target = tf.one_hot(tf.cast(target, dtype=tf.int32), 2)

# Cross Entropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(one_hot_target * tf.log(tf.clip_by_value(MLP,1e-10,1.0)), reduction_indices=[1]))

# Mean Squared Error
mean_squared_error = tf.sqrt(tf.reduce_mean(tf.square(target - MLP[:,1])))


#cost_function = mean_squared_error
cost_function = cross_entropy

#-----------------------
#REGULARIZATION
#-----------------------
#cost_function += 0.0005 * regularizers

#-----------------------
#OPTIMIZATION METHOD
#-----------------------

# Gradient Descent with Momentum
GDM_optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)

# Adam Algorithm
Adam_optimizer = tf.train.AdamOptimizer()

#training_algorithm = GDM_optimizer.minimize(cost_function)
training_algorithm = Adam_optimizer.minimize(cost_function)


<h1>MLP Training</h1>

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

n_minibatches = int(np.shape(X_train)[0] / minibatch_size) 

cost_valid_history = []
cost_train_history = []

prev_cost_valid = 10.0
validation_checks = 0
max_validation_checks = 15

for epoch in range(n_epochs):
    for i_mb in range(n_minibatches):
        a,b = i_mb*minibatch_size, (i_mb+1)*minibatch_size
        X_minibatch = X_train[a:b,:]
        Y_minibatch = Y_train[a:b]
        sess.run(training_algorithm,feed_dict={MLP_input: X_minibatch, target: Y_minibatch})
        
    cost_valid = sess.run(cost_function,feed_dict={MLP_input: X_vali, target: Y_vali})
    cost_valid_history += [cost_valid]
    
    cost_train = sess.run(cost_function,feed_dict={MLP_input: X_train, target: Y_train})
    cost_train_history += [cost_train]
    
    if prev_cost_valid < cost_valid:
        validation_checks += 1
    else:
        validation_checks = 0
        prev_cost_valid = cost_valid
        
    print("Epoch: %d/%d, Training cost: %f, Validation cost: %f, Validation checks: %d/%d" %(epoch+1, n_epochs,cost_train,cost_valid,validation_checks,max_validation_checks))
    
    if cost_train <= target_cost:
        print 'Target cost reached'
        break
    if validation_checks >= max_validation_checks:
        print 'Early stopping'
        break
    
    
print ''
    
fig = plt.figure(figsize=(6, 4))
plt.plot(cost_valid_history, label="Validation")
plt.plot(cost_train_history, label="Training")
plt.xlabel("Epoch")
plt.ylabel("Cost history")
plt.grid()
plt.legend()

<h1>MLP Testing</h1>

In [ ]:
print 'Training results:'
predicted_training_labels = sess.run(MLP[:,1],feed_dict={MLP_input: X_train})
[[VN,FP],[FN,VP]]=confusion_matrix(Y_train.astype(bool),(predicted_training_labels>0.5).astype(bool)).astype(float)
print 'VP: %d, VN: %d, FP: %d, FN: %d' %(VP,VN,FP,FN)
print 'Porcentaje de clasificaciones correctas: %%%f' %(100.0*(VP+VN)/(VP+VN+FP+FN))
print 'Precision: %%%f' %(100.0*VP/(VP+FP))
print 'Recall: %%%f' %(100.0*VP/(VP+FN))
print ''

print 'Validation results:'
predicted_validation_labels = sess.run(MLP[:,1],feed_dict={MLP_input: X_vali})
[[VN,FP],[FN,VP]]=confusion_matrix(Y_vali.astype(bool),(predicted_validation_labels>0.5).astype(bool)).astype(float)
print 'VP: %d, VN: %d, FP: %d, FN: %d' %(VP,VN,FP,FN)
print 'Porcentaje de clasificaciones correctas: %%%f' %(100.0*(VP+VN)/(VP+VN+FP+FN))
print 'Precision: %%%f' %(100.0*VP/(VP+FP))
print 'Recall: %%%f' %(100.0*VP/(VP+FN))
print ''

print 'Test results:'
predicted_test_labels = sess.run(MLP[:,1],feed_dict={MLP_input: X_test})
[[VN,FP],[FN,VP]]=confusion_matrix(Y_test.astype(bool),(predicted_test_labels>0.5).astype(bool)).astype(float)
print 'VP: %d, VN: %d, FP: %d, FN: %d' %(VP,VN,FP,FN)
print 'Porcentaje de clasificaciones correctas: %%%f' %(100.0*(VP+VN)/(VP+VN+FP+FN))
print 'Precision: %%%f' %(100.0*VP/(VP+FP))
print 'Recall: %%%f' %(100.0*VP/(VP+FN))
print ''

fpr, tpr, th = roc_curve(Y_test, predicted_test_labels)
    
plt.figure(figsize=(10, 4))
plt.subplot2grid((1,2),(0,0))
plt.plot(fpr, tpr, linewidth=2, alpha=0.5)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.grid()
plt.title('ROC Curve')

plt.subplot2grid((1,2),(0,1))
plt.plot(fpr, 1.0 - tpr, linewidth=2, alpha=0.5)
plt.xlabel("False Positive Rate")
plt.ylabel("False Negative Rate")
plt.yscale('log')
plt.xscale('log')
plt.grid()
plt.title('DET Curve')

<h1>Random Forest Classification</h1>

In [ ]:
classifier = RandomForestClassifier(n_estimators=100, max_depth=None)
classifier.fit(X_train, Y_train)

print 'Training results:'
predicted_training_labels = classifier.predict_proba(X_train)[:,1]
[[VN,FP],[FN,VP]]=confusion_matrix(Y_train.astype(bool),(predicted_training_labels>0.5).astype(bool)).astype(float)
print 'VP: %d, VN: %d, FP: %d, FN: %d' %(VP,VN,FP,FN)
print 'Porcentaje de clasificaciones correctas: %%%f' %(100.0*(VP+VN)/(VP+VN+FP+FN))
print 'Precision: %%%f' %(100.0*VP/(VP+FP))
print 'Recall: %%%f' %(100.0*VP/(VP+FN))
print ''

print 'Test results:'
predicted_test_labels = classifier.predict_proba(X_test)[:,1]
[[VN,FP],[FN,VP]]=confusion_matrix(Y_test.astype(bool),(predicted_test_labels>0.5).astype(bool)).astype(float)
print 'VP: %d, VN: %d, FP: %d, FN: %d' %(VP,VN,FP,FN)
print 'Porcentaje de clasificaciones correctas: %%%f' %(100.0*(VP+VN)/(VP+VN+FP+FN))
print 'Precision: %%%f' %(100.0*VP/(VP+FP))
print 'Recall: %%%f' %(100.0*VP/(VP+FN))
print ''

fpr, tpr, th = roc_curve(Y_test, predicted_test_labels)
    
plt.figure(figsize=(10, 4))
plt.subplot2grid((1,2),(0,0))
plt.plot(fpr, tpr, linewidth=2, alpha=0.5)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.grid()
plt.title('ROC Curve')

plt.subplot2grid((1,2),(0,1))
plt.plot(fpr, 1.0 - tpr, linewidth=2, alpha=0.5)
plt.xlabel("False Positive Rate")
plt.ylabel("False Negative Rate")
plt.yscale('log')
plt.xscale('log')
plt.grid()
plt.title('DET Curve')
